we wanna aggregate all datasets in one clean data set 
### columns are : 
company         : 1-ONCF,2-CTM,3-TAXI,4-Covoiturage

from_lat        : latitude of the origine city

from_lan        : langitude of the origine city

to_lat          : latitude of the destination city

to_lan          : langitude of the destination city

distance        : distance between origin and destination

duration        : traveling duration

driver_sex      : driver sex 0 if unkown(for example train...)

driver_age      : driver age 0 if unkown

price           : travling cost


In [2]:
import numpy as np
import requests
import json
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import re
import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression

In [7]:
def getLocationOfCity(city):
    api_key = '966a766bb88e4cb5a5f78ffd9ec99208'
    req = requests.get('https://api.opencagedata.com/geocode/v1/json?q='+city+'&key='+api_key)
    js = json.loads(req.text)
    rep = None
    for res in js['results']:
        if res['formatted'].split(',')[-1] == " Morocco":
            rep = res['geometry'] 
            break
    return rep

In [8]:
def distance(locationA,locationB):
    R = 6373.0

    lat1 = radians(locationA['lat'])
    lon1 = radians(locationA['lng'])
    lat2 = radians(locationB['lat'])
    lon2 = radians(locationB['lng'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [9]:
oncfDF = pd.read_csv("oncfTrips.csv",delimiter=";",encoding='utf-8')
ctmDF = pd.read_csv("CTMTrips.csv",delimiter=";",encoding='latin-1')
covDF = pd.read_csv("COVTrips.csv",delimiter=";",encoding='latin-1')

In [10]:
cities = list(oncfDF['from'])
cities.extend(list(oncfDF['to']))
cities.extend(list(ctmDF['from']))
cities.extend(list(ctmDF['to']))
cities.extend(list(covDF['from']))
cities.extend(list(covDF['to']))

In [11]:
cities = list(set(cities))
cities = [city.upper() for city in cities]
cities = list(set(cities))

In [12]:
locations = {}
for city in cities:
    locations[city] = getLocationOfCity(city)

In [13]:
locations['SIDI SLIMANE MEDINA']={'lat': 34.2737827,'lng':-5.9806831}
locations['KELAA  DES  SRAGHNAS']={'lat' :32.0510291,'lng':-7.4206207}
locations['GUELMIMA']={'lat' :31.6844986,'lng':-4.979725}
locations['AEROPORT MED V']={'lat' :33.3699749,'lng':-7.5879118}

In [14]:
def duration(time1,time2):
    FMT = '%H:%M'
    H1 = time1.split(':')[0]
    H2 = time2.split(':')[0]
    date1 = datetime.datetime.strptime(time1,FMT)
    date2 = datetime.datetime.strptime(time2,FMT) 
    if int(H2) < int(H1):
        date2 += timedelta(days=1)
    diff = date2 - date1
    return diff.seconds / 60

In [43]:
tfrom = oncfDF['from']
tto = oncfDF['to']
tdepart = oncfDF['depart_time']
tarrival = oncfDF['arrival_time']
tprice = oncfDF['price']
data = []
for i in range(len(tfrom)):
    data.append([1,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
               tdepart[i],duration(tdepart[i],tarrival[i]),1,0,int(tprice[i][:-3])])
    
    

In [44]:
tfrom = ctmDF['from']
tto = ctmDF['to']
tdepart = ctmDF['depart_time']
tarrival = ctmDF['arrival_time']
tprice = ctmDF['price']
tprice.fillna((tprice.mean()), inplace=True)
for i in range(len(tfrom)):
    data.append([2,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
                tdepart[i],duration(tdepart[i],tarrival[i]),1,0,int(tprice[i])])

In [17]:
len(data)

19139

In [45]:
tfrom = covDF['from']
tto = covDF['to']
tprice = covDF['price']
tsex = covDF['driver_sex'].replace({"Homme":1,"Femme":2})
tage = covDF['driver_age']
tdepart = covDF['time']
for i in range(len(tfrom)):
    data.append([3,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
                tdepart[i],
               durationFromLoc(locations[tfrom[i].upper()],locations[tto[i].upper()]),
                tsex[i],tage[i],int(tprice[i])])

In [18]:
X = np.array([86.9,52.8,59.7,64.2,41.3,78.2,64.7,141]).reshape(-1,1)
y = np.array([63,44,68,48,39,57,54,97])
duration_model = LinearRegression().fit(X, y)
duration_model.score(X, y)

0.8443326971108013

In [25]:
def durationFromLoc(locationA,locationB):
    d = distance(locationA,locationB)
    return duration_model.predict([[d]])[0]  

In [26]:
durationFromLoc(locations['MEKNES'],locations['FES'])

45.86008621137471

In [29]:
print(data)

[[1, 33.3699749, -7.5879118, 34.2296268, -6.528318, 136.87394176462, 47.0, 1, 0, 66], [1, 33.3699749, -7.5879118, 34.2296268, -6.528318, 136.87394176462, 90.0, 1, 0, 50], [1, 33.3699749, -7.5879118, 34.2296268, -6.528318, 136.87394176462, 47.0, 1, 0, 66], [1, 33.3699749, -7.5879118, 34.2296268, -6.528318, 136.87394176462, 90.0, 1, 0, 50], [1, 33.3699749, -7.5879118, 34.2296268, -6.528318, 136.87394176462, 47.0, 1, 0, 66], [1, 33.3699749, -7.5879118, 34.2296268, -6.528318, 136.87394176462, 90.0, 1, 0, 50], [1, 33.3699749, -7.5879118, 33.940302, -5.208052, 229.28611066151262, 253.0, 1, 0, 158], [1, 33.3699749, -7.5879118, 33.940302, -5.208052, 229.28611066151262, 250.0, 1, 0, 180], [1, 33.3699749, -7.5879118, 32.235967, -7.946685, 130.51896598247203, 160.0, 1, 0, 128], [1, 33.3699749, -7.5879118, 32.235967, -7.946685, 130.51896598247203, 160.0, 1, 0, 144], [1, 33.3699749, -7.5879118, 32.235967, -7.946685, 130.51896598247203, 160.0, 1, 0, 128], [1, 33.3699749, -7.5879118, 33.2676746, -7.5

In [32]:
X = np.array([202,54,91,119,60.2,67.5,63.9,24.2,78.1,188,28.8,115,44,126,124,154,85.4,25.3,51,112,128,93.8]).reshape(-1,1)
y = np.array([75,25,35,45,28,25,25,8,35,80,12,50,15,60,55,70,35,10,40,60,50,70])
price_model = LinearRegression().fit(X, y)
price_model.score(X, y)

0.8412497123562451

In [46]:
for a in locations.values():
    for b in locations.values():
        if a!=b:
            data.append([4,a['lat'],a['lng'],b['lat'],b['lng'],distance(a,b),0,durationFromLoc(a,b),1,0,price_model.predict([[distance(a,b)]])[0]])
    

In [47]:
len(data)

28395

In [48]:
pd.DataFrame(data).to_csv('dataset.csv',sep=';',header=["company" ,"from_lat" ,"from_lan" ,"to_lat" ,"to_lan" ,"distance" ,"depart","duration" ,"driver_sex" ,"driver_age" ,"price"],index=False)

In [5]:
def f(x):
    x = int(x.split(":")[0])
    if (x > 4) and (x <= 8):
        return 1
    elif (x > 8) and (x <= 12 ):
        return 2
    elif (x > 12) and (x <= 16):
        return 3
    elif (x > 16) and (x <= 20) :
        return 4
    elif (x > 20) and (x <= 24):
        return 5
    elif (x <= 4):
        return 0
def g(x):
    if x == 'M':
        return 1
    return 2

def h(x):
    p = ['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
    for a,b in enumerate(p):
        if x  == b:
            return a
    

In [67]:
trips = pd.read_csv('../dataset/dataset.csv',delimiter=";",encoding='utf-8')
users = pd.read_csv('../dataset/users.csv',delimiter=";",encoding='utf-8')
users.head()

,id,age,sex,profition
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [68]:
trips["session"] = trips['depart'].apply(f)
trips = trips.drop(axis=1,columns=['depart'])
users['profition'] = users['profition'].apply(h)
users['sex'] = users['sex'].apply(g)

In [12]:
users_types = []
for i in range(0,22):
    tmp = users.loc[users['profition']==i]
    tmp1 = tmp.loc[tmp['sex'] == 1]
    tmp2 = tmp.loc[tmp['sex'] == 2]
    users_types.extend([tmp1,tmp2])
    

In [77]:
n= np.random.randint(500,701)
dd = trips.sample(n,random_state=1)
for j in range(1,5):
    tmp = dd.loc[dd['company']==j]
    p = len(tmp)
    for k,l in zip(range(p),tmp['id']):
        print(k,l)

0 7858
1 11005
2 9376
3 12017
4 6391
5 10797
6 4423
7 3067
8 8933
9 12281
10 11697
11 2078
12 12582
13 11096
14 4158
15 11872
16 11861
17 10809
18 5036
19 4253
20 4172
21 4030
22 6190
23 10274
24 1831
25 6344
26 6619
27 11029
28 12768
29 6847
30 6294
31 242
32 11383
33 6607
34 10550
35 5235
36 4845
37 9029
38 5034
39 407
40 3271
41 5116
42 4203
43 6868
44 10239
45 8347
46 4494
47 13465
48 4344
49 1548
50 10325
51 1633
52 10115
53 2543
54 419
55 8388
56 4999
57 8781
58 2512
59 7652
60 12484
61 4954
62 9877
63 1918
64 330
65 12048
66 12024
67 5199
68 6075
69 2052
70 4593
71 8553
72 1202
73 5164
74 13141
75 1239
76 482
77 5921
78 3719
79 5655
80 13438
81 3608
82 11576
83 2416
84 1288
85 7870
86 3832
87 2369
88 1282
89 6161
90 7533
91 4383
92 3563
93 5029
94 6091
95 3069
96 8829
97 11480
98 12457
99 1513
100 2741
101 4776
102 5754
103 3351
104 7042
105 9429
106 5997
107 183
108 292
109 1988
110 4314
111 7627
112 12716
113 10030
114 3142
115 5766
116 2532
117 5004
118 1463
119 8669
120 1119

In [78]:
hist = []
m = len(users)
for i in range(m):
    n = np.random.randint(500,701)
    dd = trips.sample(n,random_state=1)
    for j in range(1,5):
        tmp = dd.loc[dd['company']==j]
        p = len(tmp)
        if j == 1:
            security = np.full(shape=p,fill_value=1)
            trafic = np.full(shape=p,fill_value=0)
            retard = np.round(np.random.normal(0.3, 0.1, p) * 60)
        else :
            security = np.round(np.random.normal(0.55, 0.1, p) * 1)
            trafic = np.round(np.random.normal(0.4, 0.1, p) * 1)
            retard = np.round(np.random.normal(0.3, 0.1, p) * 120)
        comfort = np.round(np.random.normal(0.55, 0.1, p) * 1)
        speed = np.round(np.random.normal(0.5, 0.2, p) * 2)
        for k,l in zip(range(p),tmp['id']):
            hist.append([users['id'][i],tmp['id'][l],comfort[k],speed[k],trafic[k],security[k],retard[k]])
            
            
       

In [79]:
pd.DataFrame(hist).to_csv('history.csv',sep=';',header=["user_id","trip_id","comfort","speed","trafic","security","retard"],index= False)

In [80]:
hist = pd.read_csv('history.csv',delimiter=";",encoding='utf-8')

In [84]:
len(hist)

565476

In [64]:
import collections
mu, sigma = 0.5, 0.2 # mean and standard deviation
a = np.round(np.random.normal(mu, sigma, 1000) * 2)
collections.Counter(a)

Counter({2.0: 118, 1.0: 787, 0.0: 95})